In [7]:
import numpy as np
import pandas as pd
import time

# Load the data

For $k = 0, 1, 2$ we have the following files:
* Xtrk.csv - the training sequences.
* Xtek.csv - the test sequences.
* Ytrk.csv - labels for the training sequences

In [2]:
Xtr0_mat100 = np.genfromtxt("data/Xtr0_mat100.csv", delimiter='')
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1_mat100 = np.genfromtxt("data/Xtr1_mat100.csv", delimiter='')
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2_mat100 = np.genfromtxt("data/Xtr2_mat100.csv", delimiter='')
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)


In [3]:
def accuracy(y_true,y_pred):
    n = y_true.shape[0]
    predictions = np.zeros(n)
    predictions[y_pred >= 0.5] = 1
    return np.sum(y_true == predictions) / n

# Implementing some kernels

## Linear Kernel

In [4]:
def linear_kernel(X):
    return(X @ X.T)

## Gaussian Kernel

In [20]:
def gaussian(x,y, sigma):
    exp_term = np.linalg.norm(x-y)**2 /(2*sigma)
    return(np.exp(-exp_term))

# Naive computation of the gaussian kernel that can be easily improved
def gaussian_kernel(X,sigma):
    n = X.shape[0]
    K = np.eye(n) # One along the diagonals because K(x,x) = exp(0) = 1
    for i in range(n):
        for j in range(i+1,n):
            val = gaussian(X[i], X[j], sigma)
            K[i,j] = val
            K[j,i] = val
    return(K)

# Idea : save some time by dividing by 2*sigma and applying the exponential to the matrix of pairwise distances
# (computed on a loop)
def gaussian_kernel2(X,sigma):
    n = X.shape[0]
    K = np.zeros((n,n)) # zero along the diagonals 
    for i in range(n):
        for j in range(i+1,n):
            val = np.linalg.norm(X[i]-X[j])
            K[i,j] = val
            K[j,i] = val
    K = np.exp((-K**2)/(2*sigma))
    return(K)

# Idea : efficient computation of the pairwise distances
def gaussian_kernel3(X,sigma):
    K = ((X[:, :, None] - X[:, :, None].T) ** 2).sum(1)
    K = np.exp((-K)/(2*sigma))
    return(K)
    

### Check that all three methods have the same output

In [52]:
X = Xtr0_mat100[:1000]

In [53]:
start_time = time.time()
K1 = gaussian_kernel(X, 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.411180019378662 seconds ---


In [54]:
start_time = time.time()
K2 = gaussian_kernel2(X, 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 7.064807891845703 seconds ---


In [55]:
start_time = time.time()
K3 = gaussian_kernel3(X,1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.8858790397644043 seconds ---


In [25]:
np.all(K1 == K2)

True

In [47]:
np.all(K1 == K3)

False

In [48]:
K_diff = K1 - K3
print(np.linalg.norm(K_diff))

1.62449569005189e-14


In [51]:
for i in range(1000):
    if K_diff[0,i] != 0:
        print(f"Value for K1 : {K1[0,i]}, Value for K3: {K3[0,i]}, value difference : {K_diff[0,i]}")

Value for K1 : 0.9907097748987076, Value for K3: 0.9907097748987077, value difference : -1.1102230246251565e-16
Value for K1 : 0.983944092235294, Value for K3: 0.9839440922352941, value difference : -1.1102230246251565e-16
Value for K1 : 0.9870878669144311, Value for K3: 0.9870878669144312, value difference : -1.1102230246251565e-16
Value for K1 : 0.9883715419167118, Value for K3: 0.9883715419167117, value difference : 1.1102230246251565e-16
Value for K1 : 0.9904757028915513, Value for K3: 0.9904757028915512, value difference : 1.1102230246251565e-16
Value for K1 : 0.9907097748987076, Value for K3: 0.9907097748987077, value difference : -1.1102230246251565e-16
Value for K1 : 0.9904757028915512, Value for K3: 0.9904757028915513, value difference : -1.1102230246251565e-16
Value for K1 : 0.9904757028915512, Value for K3: 0.9904757028915513, value difference : -1.1102230246251565e-16
Value for K1 : 0.9904757028915513, Value for K3: 0.9904757028915512, value difference : 1.1102230246251565e

We can see that the differnce comes at 1e-16. 
Given the norm of the difference of K1 and K3, we can consider that both matrices are equal so it is okay to use the third method which is considerably faster.

## Polynomial Kernel

## Kernel Ridge Regression

* Consider RKHS $\mathcal H$, associated to a p.d. kernel K on $\mathcal X$
* Let $y = (y_1, \dots, y_n)^T \in \mathbb R ^n$
* Let $\alpha = (\alpha_1, \dots, \alpha_n)^T \in \mathbb R ^n$
* Let $K$ be the $n\times n$ Gram Matrix such that $K_{i,j} = K(x_i, x_j)$
* We can then write
$$
(\hat f(x_1), \dots, \hat f(x_n))^T = K\alpha
$$
* The norm is $||\hat f||^2_{\mathcal H} = \alpha^T K \alpha$
* KRR $\leftrightarrow \text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n} (K\alpha - y)^T(K\alpha - y) + \lambda \alpha^T K \alpha$
* Solution for $\lambda > 0$:
$$
\alpha = (K+\lambda nI)^{-1}y
$$


In [56]:
def KRR(K, y, Kval, yval, lambd):
    """
    takes the kernel matrix as an input and computes the MSE and the predictions for each value in lambd (list)
    """
    assert K.shape[0] == y.shape[0]
    assert len(lambd) > 0
    n = K.shape[0]
    
    loss = []
    acc = []
    
    loss_val = []
    acc_val = []
    alphas = []
    
    for l in lambd:
        
        assert l >= 0
        # find the parameter alpha
        alpha = np.linalg.solve((K + l*n*np.eye(n)), y)
        # predict
        
        loss_lambda = MSE(K, y, l, alpha)
        acc_lambda = accuracy(y,K@alpha)
        
        loss_lambdaval = MSE(Kval, yval, l, alpha)
        acc_lambdaval = accuracy(yval,Kval@alpha)

        print(f"***********lambda = {l}***********")
        print(f"Training: loss = {loss_lambda:.4f}, accuracy = {acc_lambda:.6f}")
        print(f"Validation: loss = {loss_lambdaval:.4f}, accuracy = {acc_lambdaval:.6f}")
        
        loss += [loss_lambda]
        acc += [acc_lambda]
        
        loss_val += [loss_lambdaval]
        acc_val += [acc_lambdaval]
        
        
        alphas +=[alpha]
        
    return(alphas, loss, acc, loss_val, acc_val)
    

In [57]:
def MSE(K, y, lambd, alpha):
    n = y.shape[0]
    data_term = (np.linalg.norm(np.dot(K, alpha.reshape(-1,1)) - y)**2)/n
    reg_term = alpha @ K @ alpha
    return(data_term + lambd * reg_term)

## Kernel Logistic Regression

- Binary Classificaiton setup: $\mathcal Y = \{-1, 1\}$
- $\mathcal l_{\text{logistic}}(f(x),y) = -\log p(y|f(x)) = \log(1 + e^{-yf(x)})$ where $p(y|f(x)) = \sigma(y(f(x))$

Objective:
\begin{align*}
\hat f &= \text{argmin}_{f\in \mathcal H} \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_if(x_i)}) + \frac{\lambda}{2}||f||^2_{\mathcal H}\\
\alpha &= \text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_i[K\alpha]_i}) + \frac{\lambda}{2} \alpha^T K \alpha
\end{align*}

We define the following fonctions and vectors:
* $\mathcal l _\text{logistic}(u) = \log(1+e^{-u})$
* $\mathcal l' _\text{logistic}(u) = -\sigma(-u)$
* $\mathcal l'' _\text{logistic}(u) = \sigma(u)\sigma(-u)$

* for $i = 1, \dots, n$, $P_i(\alpha) = \mathcal l' _\text{logistic}(y_i[K\alpha]_i)$
* for $i = 1, \dots, n$, $W_i(\alpha) = \mathcal l'' _\text{logistic}(y_i[K\alpha]_i)$




\begin{align*}
J(\alpha) &= \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_i[K\alpha]_i}) + \frac{\lambda}{2} \alpha^T K \alpha\\
\nabla J(\alpha) &= \frac{1}{n} KP(\alpha) y + \lambda K \alpha \quad \text{where } P(\alpha) = \text{diag}(P_1(\alpha), \dots, P_n(\alpha))\\
\nabla^2 J(\alpha) &= \frac{1}{n}KW(\alpha)K+\lambda K \quad \text{where } W(\alpha) = \text{diag}(W_1(\alpha), \dots, W_n(\alpha))
\end{align*}

We are interested in the quadratic approximation of $J$ near a point $\alpha_0$:
\begin{align*}
J_q(\alpha) &= J(\alpha_0) + (\alpha - \alpha_0)^T \nabla J(\alpha_0) + \frac{1}{2} (\alpha - \alpha_0)^T \nabla^2 J(\alpha_0)(\alpha - \alpha_0)\\
2J_q(\alpha) &= -\frac{2}{n} \alpha^T KW(K\alpha_0-W^{-1}Py)+\frac{1}{n}\alpha^TKWK\alpha+ \lambda\alpha^TK\alpha +C\\
&= \frac{1}{n} (K\alpha - z)^TW(K\alpha - z) + \lambda\alpha^TK\alpha + C \quad \text{where} z = K\alpha_0 - W^{-1} P y
\end{align*}

The WKRR problem is presented as:
$$
\text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n}(K\alpha - y)^TW(K\alpha - y) + \lambda \alpha^TK\alpha
$$
and has as :
$$
\alpha = W^{1/2} (W^{1/2}KW^{1/2}+n\lambda I)^{-1} W^{1/2}y
$$

So, in order to solve KRL, we use IRLS on a WKRR problem until convergence:
$$\alpha^{t+1} \gets \text{solveWKRR}(K, W^t, z^t)$$
With the updates for $W^t$ and $z^t$ from $\alpha^t$ are:
- $m_i \gets [K\alpha^t]_i$
- $P_i^t \gets -\sigma(-y_im_i)$
- $W_i^t \gets \sigma(m_i)\sigma(-m_i)$
- $z_i^t \gets m_i + y_i / \sigma(-y_im_i)$

In [58]:
def solveWKRR(K,W,z,y,lambd):
    
    assert np.all(W >= 0)
    
    W_sq = np.sqrt(W)
    n = K.shape[0]
    inv_matrix = np.linalg.solve((W_sq @ K @ W_sq + n * lambd * np.eye(n)), W_sq @ y)
    alpha = W_sq @ inv_matrix
    
    return alpha
    

In [59]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def logistic_loss(y_true, y_pred):
    n = y_true.shape[0]
    log_term = np.log(sigmoid(y_true*y_pred))
    return(-np.sum(log_term)/n)
    

In [60]:
def KLR(K, y, Kval, yval, lambd, maxIter = 100, tresh = 1e-8):
    
    # initialize the values
    assert K.shape[0] == y.shape[0]
    n = K.shape[0]
    
    loss = []
    acc = []
    
    loss_val = []
    acc_val = []
    
    
    alphas = []
    
    for l in lambd :
        cnt = 0
        
        P_t, W_t = np.eye(n), np.eye(n)
        z_t = K@ np.ones(n) - y
        alpha_t = np.ones(n)
        diff_alpha = np.inf


        while (diff_alpha > tresh) and (cnt < maxIter):

            old_alpha = alpha_t
            alpha_t = solveWKRR(K, W_t, z_t, y, l)

            m_t = K@alpha_t
            sigma_m = sigmoid(m_t)
            sigma_my = sigmoid(-y*m_t)

            P_t = - np.diag(sigma_my)
            W_t = np.diag(sigma_m * (1-sigma_m))

            z_t = m_t - (P_t@y)/(sigma_m * (1-sigma_m))

            diff_alpha = np.linalg.norm(alpha_t - old_alpha)
            cnt+=1
            if cnt % 10 == 0:
                print(l, cnt)
        
        loss_lambda = logistic_loss(y, K@alpha_t)
        acc_lambda = accuracy(y,K@alpha_t)
        
        loss_lambdaval = logistic_loss(yval, Kval@alpha_t)
        acc_lambdaval = accuracy(yval,Kval@alpha_t)

        
        print(f"***********lambda = {l}***********")
        print(f"Training: loss = {loss_lambda:.4f}, accuracy = {acc_lambda:.6f}")
        print(f"Validation: loss = {loss_lambdaval:.4f}, accuracy = {acc_lambdaval:.6f}")
        
        
        loss += [loss_lambda]
        acc += [acc_lambda]
        
        loss_val += [loss_lambdaval]
        acc_val += [acc_lambdaval]
        
        alphas +=[alpha_t]
        
    return(alphas, loss, acc, loss_val, acc_val)
        

## Support Vector Machine approach (SVM)

- Binary Classificaiton setup: $\mathcal Y = \{-1, 1\}$
- $\mathcal l_{\text{hinge}}(f(x),y) = \max(1- y f(x), 0)$

Objective:
\begin{align*}
\hat f &= \text{argmin}_{f\in \mathcal H} \frac{1}{n} \sum_{i=1}^n \max(1- y_i f(x_i), 0) + \lambda||f||^2_{\mathcal H}\\
\alpha &= \text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n} \sum_{i=1}^n \max(y_i[K\alpha]_i, 0) + \lambda \alpha^T K \alpha
\end{align*}

It is a convex optimization problem but the objective is not smooth.

By introducing additional slack variables $\xi_i$, the problem's objective becomes smooth but it is not the case for the constraints anymore. Let us solve the dual formulation instead (which is sparse, leading to faster algorithms). 

The dual can be rewritten as a quadratic minimization under box constraints : 

\begin{align*}
\min_{\alpha \in \mathbb R^n} \frac{1}{2} \alpha^TK\alpha - \alpha^T y\\
\text{s.t. }  0\leq y_i\alpha_i\leq \frac{1}{2\lambda n}
\end{align*}


We will solve it using CVXOpt tools

In [61]:
def hinge_loss(y_true, y_pred):
    n = y_true.shape[0]
    term = np.maximum(1-y_true*y_pred, 0)
    return(np.sum(term)/n)

In [62]:
## Je pense qu'une plus belle façon de faire serait de créer des fonctions de 
## kernel(X, sigma) et de les appeler avec en paramètres (X_train ou X_val) selon si on 
## fait le training ou la validation, pour pas avoir à garder en mémoire les kernels train/val
## comme on le fait jusqu'à maintenant
## Mais bon, là il est 2h47 du matin, j'ai un peu la flemme et j'imagine que toi aussi,
## ça marche déjà bien comme ça ^^

def _gaussian_kernel(sigma=1):
    """
    Prepares a Gaussian RBF kernel using the provided sigma.

    Returns:
    --------
    kernel_function: A callable to the Gaussian RBF kernel function.

    """
    gamma = -1 / (2 * sigma ** 2)
    kernel_function = lambda X, y: np.exp(gamma * np.square(X[:, np.newaxis] - y).sum(axis=2))
    return kernel_function

In [63]:
from cvxopt import matrix, solvers

def SVM(K, y, K_val, y_val, lambd):
    # initialize the values
    assert K.shape[0] == y.shape[0]
    n = K.shape[0]
    
    y_preds, y_preds_val = [], []
    losses, losses_val = [], []
    accuracies, accuracies_val = [], []
    alphas = []
    
    
    for l in lambd :

            ## Solving dual using CVXOpt
        P = matrix(K)
        q = matrix(-y)
        D = np.diag(-y)
        G = matrix(np.vstack((D,-D)))
        h = matrix(np.concatenate((np.zeros(n), 1/(2*l) * np.ones(n)), axis=0))
        solvers.options['show_progress'] = False
        sol=solvers.qp(P, q, G, h)
        alpha = sol['x']
        alpha = np.reshape(alpha,-1)               

            ## predictions
        # training
        pred_l = K @alpha
        y_preds += [pred_l]
        loss_l = hinge_loss(y, pred_l)
        acc_l = accuracy(y, pred_l)

        
        # validation
        pred_l_val = K_val@alpha
        y_preds_val += [pred_l_val]
        loss_l_val = hinge_loss(y_val, pred_l_val)
        acc_l_val = accuracy(y_val,pred_l_val)
        

        print(15*"-", f" lambda = {l} ", 15*"-")
        print(f"Training: loss = {loss_l:.4f}, accuracy = {acc_l:.6f}")
        print(f"Validation: loss = {loss_l_val:.4f}, accuracy = {acc_l_val:.6f}")
        
        losses += [loss_l]
        accuracies += [acc_l]
        
        losses_val += [loss_l_val]
        accuracies_val += [acc_l_val]
    
        alphas +=[alpha] 
        
    return(alphas, losses, accuracies, losses_val, accuracies_val)
            

# Testing the accuracy

## Splitting data

In [64]:
from sklearn.model_selection import train_test_split

Xtr0, Xval0, ytr0, yval0 = train_test_split(Xtr0_mat100, Ytr0, test_size=0.5, random_state=42)
Xtr1, Xval1, ytr1, yval1 = train_test_split(Xtr1_mat100, Ytr1, test_size=0.5, random_state=42)
Xtr2, Xval2, ytr2, yval2 = train_test_split(Xtr2_mat100, Ytr2, test_size=0.5, random_state=42)

## Create the kernel matrices

In [65]:
K_tr0 = gaussian_kernel3(Xtr0,0.5)
K_tr1 = gaussian_kernel3(Xtr1,0.5)
K_tr2 = gaussian_kernel3(Xtr2,0.5)

K_val0 = gaussian_kernel3(Xval0,0.5)
K_val1 = gaussian_kernel3(Xval1,0.5)
K_val2 = gaussian_kernel3(Xval2,0.5)

## Testing KRR

In [66]:
lambdas = [0] + [10**i for i in range(-10,2)]
print("*************KRR for dataset 0*************\n")
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = KRR(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas)
print("*************KRR for dataset 1*************\n")
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = KRR(K_tr1, ytr1[:,1], K_val1, yval1[:,1],lambdas)
print("*************KRR for dataset 2*************\n")
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = KRR(K_tr2, ytr2[:,1], K_val2, yval2[:,1],lambdas)

*************KRR for dataset 0*************

***********lambda = 0***********
Training: loss = 497.5500, accuracy = 1.000000
Validation: loss = 883887190.5092, accuracy = 0.483000
***********lambda = 1e-10***********
Training: loss = 497.3042, accuracy = 1.000000
Validation: loss = 880451639.5200, accuracy = 0.483000
***********lambda = 1e-09***********
Training: loss = 495.1282, accuracy = 1.000000
Validation: loss = 850853163.6289, accuracy = 0.484000
***********lambda = 1e-08***********
Training: loss = 476.3027, accuracy = 1.000000
Validation: loss = 640425725.6463, accuracy = 0.481000
***********lambda = 1e-07***********
Training: loss = 393.9756, accuracy = 0.999000
Validation: loss = 160132915.1740, accuracy = 0.487000
***********lambda = 1e-06***********
Training: loss = 309.6085, accuracy = 0.877000
Validation: loss = 5538134.4045, accuracy = 0.477000
***********lambda = 1e-05***********
Training: loss = 287.5814, accuracy = 0.717000
Validation: loss = 100382.3624, accuracy = 

## Testing KLR

In [353]:
lambdas = [1e-8]
print("*************KLR for dataset 0*************\n")
alphas_tr0_klr, loss_tr0_klr, acc_0_klr, loss_val0_klr, acc_val0_klr = KLR(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas, tresh=1e-5)
print("*************KLR for dataset 1*************\n")
alphas_tr1_klr, loss_tr1_klr, acc_1_klr, loss_val1_klr, acc_val1_klr = KLR(K_tr1, ytr1[:,1], K_val1, yval1[:,1], lambdas, tresh=1e-5)
print("*************KLR for dataset 2*************\n")
alphas_tr2_klr, loss_tr2_klr, acc_2_klr, loss_val2_klr, acc_val2_klr = KLR(K_tr2, ytr2[:,1], K_val2, yval2[:,1], lambdas, tresh=1e-5)


*************KLR for dataset 0*************

1e-08 10
1e-08 20
1e-08 30
1e-08 40
1e-08 50
1e-08 60
1e-08 70
1e-08 80
1e-08 90
1e-08 100
***********lambda = 1e-08***********
Training: loss = 0.5439, accuracy = 0.993000
Validation: loss = inf, accuracy = 0.485000
*************KLR for dataset 1*************



/Users/Claudia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/Claudia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


1e-08 10
1e-08 20
1e-08 30
1e-08 40
1e-08 50
1e-08 60
1e-08 70
1e-08 80
1e-08 90
1e-08 100
***********lambda = 1e-08***********
Training: loss = 0.5364, accuracy = 0.991000
Validation: loss = inf, accuracy = 0.474000
*************KLR for dataset 2*************

1e-08 10
1e-08 20
1e-08 30
1e-08 40
1e-08 50
1e-08 60
1e-08 70
1e-08 80
1e-08 90
1e-08 100
***********lambda = 1e-08***********
Training: loss = 0.5267, accuracy = 0.992000
Validation: loss = inf, accuracy = 0.493000


## Testing SVM

In [67]:
lambdas = [10**i for i in range(-10,2)]
print("************* SVM for dataset 0 *************\n")
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = SVM(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas)

print("")
print("")
print("************* SVM for dataset 1 *************\n")
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = SVM(K_tr1, ytr1[:,1], K_val1, yval1[:,1],lambdas)

print("")
print("")
print("************* SVM for dataset 2 *************\n")
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = SVM(K_tr2, ytr2[:,1], K_val2, yval2[:,1],lambdas)

************* SVM for dataset 0 *************

---------------  lambda = 1e-10  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 332.0737, accuracy = 0.478000
---------------  lambda = 1e-09  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 331.9922, accuracy = 0.478000
---------------  lambda = 1e-08  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 332.0481, accuracy = 0.478000
---------------  lambda = 1e-07  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 331.9515, accuracy = 0.478000
---------------  lambda = 1e-06  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 332.0244, accuracy = 0.478000
---------------  lambda = 1e-05  ---------------
Training: loss = 0.5350, accuracy = 1.000000
Validation: loss = 332.0678, accuracy = 0.478000
---------------  lambda = 0.0001  ---------------
Training: loss = 0.5496, accuracy = 0.995000


## Making predictions

### First create the kernels for each testing set with the chosen parameters

In [292]:
Xte0 = np.genfromtxt("data/Xte0_mat100.csv", delimiter='')
Xte1 = np.genfromtxt("data/Xte1_mat100.csv", delimiter='')
Xte2 = np.genfromtxt("data/Xte2_mat100.csv", delimiter='')

In [293]:
K_te0 = gaussian_kernel(Xte0,1)
K_te1 = gaussian_kernel(Xte1,1)
K_te2 = gaussian_kernel(Xte2,1)

In [378]:
def write_predictions_csv(test_kernels, test_alphas, path):
    
    predictions = np.zeros(3000, dtype=int)
    
    for i in range(3):
        y_pred = test_kernels[i] @ test_alphas[i]
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        
        predictions[1000*i:1000*(i+1)] = y_pred
    
    #predictions = predictions.astype(int)
    pred = pd.DataFrame({"Bound" : predictions})
    pred.to_csv(path, index=True,index_label="Id")
    print("saving predictions")
    #np.savetxt("data/Ytest_KRR.csv", predictions, header = "Id, Bound", delimiter =",")
    print("saved predictions")
    return(predictions)
        

Example

In [393]:
test_kernels = [K_te0, K_te1, K_te2]
#test_alphas = [alphas_tr0[-4], alphas_tr1[-4], alphas_tr2[-3]] # il faut choisir l'alpha associé à un bon lambda!
test_alphas = [alphas_tr0_klr[0], alphas_tr1_klr[0], alphas_tr2_klr[0]]
write_predictions_csv(test_kernels, test_alphas, path ="data/Ytest_KLR.csv")

saving predictions
saved predictions


array([0, 0, 1, ..., 0, 1, 1])

In [395]:
test_kernels = [K_te0, K_te1, K_te2]
test_alphas = [alphas_tr0[8], alphas_tr1[8], alphas_tr2[10]] # il faut choisir l'alpha associé à un bon lambda!

mmm=write_predictions_csv(test_kernels, test_alphas, path="data/Ytest_SVM.csv")

saving predictions
saved predictions
